# Character Issues

In [1]:
s = 'café'
len(s)

4

In [3]:
b = s.encode('utf8')
b

b'caf\xc3\xa9'

In [4]:
len(b)

5

In [5]:
b.decode('utf8')

'café'

# Byte Essentials

In [6]:
cafe = bytes('café', encoding='utf_8')
cafe

b'caf\xc3\xa9'

In [7]:
cafe[0]

99

In [8]:
cafe[:1]

b'c'

In [12]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [13]:
cafe_arr[-1:]

bytearray(b'\xa9')

In [17]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

In [19]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
octets = bytes(numbers)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

## Structs and Memory Views

In [20]:
import struct

In [26]:
fmt = '<3s3sHH'
with open('1st day school.jpg', 'rb') as fp:
    img = memoryview(fp.read())

In [27]:
header = img[:10]

In [36]:
bytes(header)

b'\xff\xd8\xff\xe0\x00\x10JFIF'

In [37]:
struct.unpack(fmt, header)

(b'\xff\xd8\xff', b'\xe0\x00\x10', 17994, 17993)

In [38]:
del header
del img

# Basic Encoders/Decoders